In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.0 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.8 MB/s eta 0:00:00


In [2]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.8 MB/s eta 0:00:00


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from datasets import load_dataset,Dataset,DatasetDict

In [5]:
df = pd.read_csv('/content/labeled_data.csv')

In [6]:
import ast

# Function to process the label column
def process_label(label):
    # Use ast.literal_eval to safely evaluate the content inside the brackets
    try:
        label_list = ast.literal_eval(label)
        if isinstance(label_list, list):
            if len(label_list) == 0:
                return [0]
            return label_list
    except (SyntaxError, ValueError):
        pass
    return [0]

In [7]:
# Apply the process_label function to each row in the label column
df['label'] = df['label'].apply(process_label)

# Explode the label lists into separate rows
df = df.explode('label', ignore_index=True)
df = df.drop(['Unnamed: 0'], axis=1)

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

df['label'] = df['label'].apply(lambda x: pad_sequences(maxlen=50, sequences=df['label'], padding='post', value=0))

In [ ]:
X_data = df.content
y_data = df.label

In [ ]:
# from keras.utils import to_categorical
# df['label']= df['label'].apply(lambda i: to_categorical(i, num_classes=5)).values

In [9]:
df["label"]

0       [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0,...
1       [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0,...
2       [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0,...
3       [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0,...
4       [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0,...
                              ...                        
1079    [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0,...
1080    [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0,...
1081    [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0,...
1082    [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0,...
1083    [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0,...
Name: label, Length: 1084, dtype: object

In [10]:
df['label'] = df['label'].apply(lambda x: [item for sublist in x for item in sublist])

In [11]:
df

,content,label
0,"['iPhone_14 được ra_mắt vào 09/2022 , được đán...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, ..."
1,"['iPhone_14 được ra_mắt vào 09/2022 , được đán...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, ..."
2,['Về ngoại_hình thì cũng có rất_ít sự_khác_biệ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, ..."
3,"['Nhìn_chung , về thiết_kế của iPhone_14 thì k...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, ..."
4,"['Tương_tự , thì hệ_thống_camera vẫn giữ_nguyê...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, ..."
...,...,...
1079,['Mặc dù cả hai_thiết bị_đều sở_hữu vi...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, ..."
1080,['Mặc dù cả hai_thiết bị_đều sở_hữu vi...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, ..."
1081,"['Tuy_nhiên , sau hai giờ sử dụng thì điệ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, ..."
1082,['iPhone_14 Pro_Max có thời_lượng pin_tốt ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, ..."


In [12]:
df['label'][0]

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 1,
 0,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 4,
 3,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [13]:
data = Dataset.from_pandas(df)

In [14]:
train_testvalid = data.train_test_split(test_size=0.2,seed=15)

# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5,seed=15)

In [15]:
data = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})


In [16]:
tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
checkpoint = "NlpHUST/ner-vietnamese-electra-base"

In [17]:
tokenizer.model_max_len=512

In [18]:
def tokenize(batch):
  return tokenizer(batch["content"], truncation=True,max_length=512)

tokenized_dataset = data.map(tokenize, batched=True)
tokenized_dataset

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

Map:   0%|          | 0/109 [00:00<?, ? examples/s]

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 867
    })
    test: Dataset({
        features: ['content', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 109
    })
    valid: Dataset({
        features: ['content', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 108
    })
})

In [19]:
tokenized_dataset.set_format("torch",columns=["input_ids", "attention_mask", "label"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**I still can't figure out how to tune the custom model so that it works with my data**

In [20]:
class CustomModel(nn.Module):
  def __init__(self,num_labels):
    super(CustomModel,self).__init__()
    self.num_labels = num_labels

    #Load Model with given checkpoint and extract its body
    self.model = model
    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(288,num_labels) # load and initialize weights

  def forward(self, input_ids=None, attention_mask=None,labels=None):
    #Extract outputs from the body
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

    #Add custom layers
    sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state

    logits = self.classifier(sequence_output[:,0,:].reshape(-1,288)) # calculate losses

    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

    return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=CustomModel(num_labels=5).to(device)

In [22]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size=32, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_dataset["valid"], batch_size=32, collate_fn=data_collator
)

In [23]:
from transformers import AdamW,get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

84


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:
from datasets import load_metric
metric = load_metric("f1")


<ipython-input-24-e4b2221a8929>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


In [25]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epochs * len(eval_dataloader)))


for epoch in range(num_epochs):
  model.train()
  for batch in train_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      progress_bar_train.update(1)

  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar_eval.update(1)

  print(metric.compute())

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: ignored

In [ ]:
model.eval()

test_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=32, collate_fn=data_collator
)

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()